In [1]:
using Laplacians
using SuiteSparse
using LinearAlgebra
using SparseArrays
using Statistics

In [2]:
import Laplacians.LLMatOrd
import Laplacians.approxChol
import Laplacians.LDLsolver
import Laplacians.approxchol_lapGiven
import Laplacians.LLord
import Laplacians.LDLinv
import Laplacians.LLcol

In [3]:
a = grid2(2)
la = lap(a)

4×4 SparseMatrixCSC{Float64,Int64} with 12 stored entries:
  [1, 1]  =  2.0
  [2, 1]  =  -1.0
  [3, 1]  =  -1.0
  [1, 2]  =  -1.0
  [2, 2]  =  2.0
  [4, 2]  =  -1.0
  [1, 3]  =  -1.0
  [3, 3]  =  2.0
  [4, 3]  =  -1.0
  [2, 4]  =  -1.0
  [3, 4]  =  -1.0
  [4, 4]  =  2.0

In [4]:
n = size(a)[1]
b = randn(size(la,1))
b = b.- mean(b)

4-element Array{Float64,1}:
  0.35177873890348577
  0.873679423502436
 -1.3446358004528314
  0.11917763804690962

In [10]:
a

4×4 SparseMatrixCSC{Float64,Int64} with 8 stored entries:
  [2, 1]  =  1.0
  [3, 1]  =  1.0
  [1, 2]  =  1.0
  [4, 2]  =  1.0
  [1, 3]  =  1.0
  [4, 3]  =  1.0
  [2, 4]  =  1.0
  [3, 4]  =  1.0

In [5]:
llmat = LLMatOrd(a)

LLMatOrd{Int64,Float64}(4, [2, 3, 4, 0], LLord{Int64,Float64}[LLord{Int64,Float64}(2, 0, 1.0), LLord{Int64,Float64}(3, 1, 1.0), LLord{Int64,Float64}(4, 0, 1.0), LLord{Int64,Float64}(4, 0, 1.0), LLord{Int64,Float64}(4613210992, 4613211056, 2.2792291314e-314), LLord{Int64,Float64}(4613211184, 4613211248, 2.2792292263e-314), LLord{Int64,Float64}(4613211376, 4613211440, 2.279229321e-314), LLord{Int64,Float64}(4613211568, 4613211632, 0.0)])

In [6]:
ldli = approxChol(llmat)

LDLinv{Int64,Float64}([1, 2, 3], [1, 3, 5, 6], [2, 3, 3, 4, 4], [0.5, 1.0, 0.3333333333333333, 1.0, 1.0], [0.5, 0.6666666666666667, 1.3333333333333335, 0.0])

In [7]:
y = LDLsolver(ldli, b)

4-element Array{Float64,1}:
  0.1170198223329435
  0.4957092588700175
 -0.6134483531076163
  0.0007192719046552609

In [8]:
solver = approxchol_lapGiven(a); 
y_p = solver(b)
y_p

4-element Array{Float64,1}:
  0.11701982233294347
  0.4957092588700174
 -0.6134483531076161
  0.0007192719046552608

In [9]:
norm(la*y-b)/norm(b) 

2.4319370807575973e-16

In [11]:
norm(la*y_p-b)/norm(b)

3.3893104428060695e-16